In [1]:
import pandas as pd
import numpy as np 

## Importing Datasets

In [2]:
comp_ord= pd.read_excel('D:\internship\Cointab\Company X - Order Report.xlsx')
comp_pin = pd.read_excel('D:\internship\Cointab\Company X - Pincode Zones.xlsx')
comp_sku = pd.read_excel('D:\internship\Cointab\Company X - SKU Master.xlsx')
cour_inv = pd.read_excel('D:\internship\Cointab\Courier Company - Invoice.xlsx')
cour_rate = pd.read_excel('D:\internship\Cointab\Courier Company - Rates.xlsx')
expec_result = pd.read_excel('D:\internship\Cointab\Expected_Result.xlsx',sheet_name='Calculations')

In [3]:
comp_ord.rename(columns={'ExternOrderNo': 'Order ID'}, inplace=True)
comp_ord.head()

,Order ID,SKU,Order Qty,Payment Mode,Item Price(Per Qty.)
0,2001827036,8904223818706,1,COD,233
1,2001827036,8904223819093,1,COD,233
2,2001827036,8904223819109,1,COD,233
3,2001827036,8904223818430,1,COD,470
4,2001827036,8904223819277,1,COD,233


In [4]:
comp_ord2 = comp_ord.copy()


In [5]:
comp_ord2 = pd.merge(left=comp_ord2,right=comp_sku,on='SKU',how = 'inner')
comp_ord2 = comp_ord2.sort_values(by='Order ID',ascending=True)
comp_ord2.head()

,Order ID,SKU,Order Qty,Payment Mode,Item Price(Per Qty.),Weight (g)
338,2001806210,8904223818874,1,COD,922,100
328,2001806210,8904223816214,1,COD,402,120
217,2001806226,8904223818850,2,COD,1125,240
266,2001806229,8904223818942,1,COD,286,133
216,2001806229,8904223818850,1,COD,563,240


## Obtaining Total weight by company X

In [6]:
comp_ord2['total_weight_by_comp'] = comp_ord2['Weight (g)'] * comp_ord2['Order Qty']
total_sum = comp_ord2.groupby('Order ID')['total_weight_by_comp'].sum().reset_index()
total_sum['total_weight_by_comp'] = total_sum['total_weight_by_comp']/1000

In [7]:
new_df = cour_inv[['AWB Code', 'Order ID']]
new_df = new_df.merge(right=total_sum,on='Order ID',how='inner')
new_df = new_df.merge(right= cour_inv[['Order ID','Customer Pincode','Zone']],on = 'Order ID',how='inner')
new_df.rename(columns={'Zone' :'Zone_by_cour'},inplace=True)
new_df.head()


,AWB Code,Order ID,total_weight_by_comp,Customer Pincode,Zone_by_cour
0,1091117222124,2001806232,1.302,507101,d
1,1091117222194,2001806273,0.615,486886,d
2,1091117222931,2001806408,2.265,532484,d
3,1091117223244,2001806458,0.700,143001,b
4,1091117229345,2001807012,0.240,515591,d


## Merging The Zones by Company X and Courier service 

In [8]:
new_df = new_df.merge(right= comp_pin[['Customer Pincode','Zone']].drop_duplicates(),on='Customer Pincode',how='left')
new_df.rename(columns={'Zone':'Zone_by_comp'},inplace=True)
new_df.head()

,AWB Code,Order ID,total_weight_by_comp,Customer Pincode,Zone_by_cour,Zone_by_comp
0,1091117222124,2001806232,1.302,507101,d,d
1,1091117222194,2001806273,0.615,486886,d,d
2,1091117222931,2001806408,2.265,532484,d,d
3,1091117223244,2001806458,0.700,143001,b,b
4,1091117229345,2001807012,0.240,515591,d,d


## Giving respective weight slabs by Company x and Courier as per respective given zones

In [9]:
new_df = new_df.merge(right=cour_rate[['Zone', 'Weight Slabs']],left_on='Zone_by_comp', right_on='Zone',how='left')
new_df.drop(columns='Zone',inplace=True)


In [10]:
new_df['Weight_Slab_by_comp'] = np.ceil(new_df['total_weight_by_comp'] / new_df['Weight Slabs']) * new_df['Weight Slabs']
new_df.drop(columns='Weight Slabs',inplace=True)
new_df.head()

,AWB Code,Order ID,total_weight_by_comp,Customer Pincode,Zone_by_cour,Zone_by_comp,Weight_Slab_by_comp
0,1091117222124,2001806232,1.302,507101,d,d,2.50
1,1091117222194,2001806273,0.615,486886,d,d,1.25
2,1091117222931,2001806408,2.265,532484,d,d,2.50
3,1091117223244,2001806458,0.700,143001,b,b,1.00
4,1091117229345,2001807012,0.240,515591,d,d,1.25


In [11]:
new_df = new_df.merge(right=cour_rate[['Zone', 'Weight Slabs']],left_on='Zone_by_cour', right_on='Zone',how='left')
new_df.drop(columns='Zone',inplace=True)
#Here we are also fetching the Weight given by courier service
new_df = new_df.merge(right=cour_inv[['Charged Weight','Order ID']],on= ['Order ID'],how='left')
new_df.rename(columns={'Charged Weight':'Weight_by_cour'},inplace=True)
new_df.head()

,AWB Code,Order ID,total_weight_by_comp,Customer Pincode,Zone_by_cour,Zone_by_comp,Weight_Slab_by_comp,Weight Slabs,Weight_by_cour
0,1091117222124,2001806232,1.302,507101,d,d,2.50,1.25,1.30
1,1091117222194,2001806273,0.615,486886,d,d,1.25,1.25,1.00
2,1091117222931,2001806408,2.265,532484,d,d,2.50,1.25,2.50
3,1091117223244,2001806458,0.700,143001,b,b,1.00,0.50,1.00
4,1091117229345,2001807012,0.240,515591,d,d,1.25,1.25,0.15


In [12]:
new_df['Weight_Slab_by_cour'] = np.ceil(new_df['Weight_by_cour'] / new_df['Weight Slabs']) * new_df['Weight Slabs']
new_df.drop(columns='Weight Slabs',inplace=True)
new_df.head()

,AWB Code,Order ID,total_weight_by_comp,Customer Pincode,Zone_by_cour,Zone_by_comp,Weight_Slab_by_comp,Weight_by_cour,Weight_Slab_by_cour
0,1091117222124,2001806232,1.302,507101,d,d,2.50,1.30,2.50
1,1091117222194,2001806273,0.615,486886,d,d,1.25,1.00,1.25
2,1091117222931,2001806408,2.265,532484,d,d,2.50,2.50,2.50
3,1091117223244,2001806458,0.700,143001,b,b,1.00,1.00,1.00
4,1091117229345,2001807012,0.240,515591,d,d,1.25,0.15,1.25


In [13]:
#re arranging the columns order
new_df = new_df[['AWB Code', 'Order ID','Customer Pincode', 'total_weight_by_comp', 'Weight_by_cour', 'Zone_by_comp','Zone_by_cour','Weight_Slab_by_comp','Weight_Slab_by_cour']]
new_df.head()

,AWB Code,Order ID,Customer Pincode,total_weight_by_comp,Weight_by_cour,Zone_by_comp,Zone_by_cour,Weight_Slab_by_comp,Weight_Slab_by_cour
0,1091117222124,2001806232,507101,1.302,1.30,d,d,2.50,2.50
1,1091117222194,2001806273,486886,0.615,1.00,d,d,1.25,1.25
2,1091117222931,2001806408,532484,2.265,2.50,d,d,2.50,2.50
3,1091117223244,2001806458,143001,0.700,1.00,b,b,1.00,1.00
4,1091117229345,2001807012,515591,0.240,0.15,d,d,1.25,1.25


## Obtaing the expected total forward charge by the company X

In [14]:
fwd_df = new_df[['Order ID','total_weight_by_comp', 'Zone_by_comp','Weight_Slab_by_comp']]
fwd_df = fwd_df.merge(right=cour_rate,left_on='Zone_by_comp',right_on='Zone',how='left')
fwd_df.drop(columns=['Zone'],inplace=True)
fwd_df = fwd_df.merge(right=cour_inv[['Order ID', 'Type of Shipment']],on='Order ID',how='left')
fwd_df.head()

,Order ID,total_weight_by_comp,Zone_by_comp,Weight_Slab_by_comp,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge,Type of Shipment
0,2001806232,1.302,d,2.50,1.25,45.4,44.8,41.3,44.8,Forward charges
1,2001806273,0.615,d,1.25,1.25,45.4,44.8,41.3,44.8,Forward charges
2,2001806408,2.265,d,2.50,1.25,45.4,44.8,41.3,44.8,Forward charges
3,2001806458,0.700,b,1.00,0.50,33.0,28.3,20.5,28.3,Forward charges
4,2001807012,0.240,d,1.25,1.25,45.4,44.8,41.3,44.8,Forward charges


In [15]:
# Define a custom function to calculate the total forward charges based on the Shipment type
def calculate_total_forward_charges(row):
    weight_slabs_by_comp = row['Weight_Slab_by_comp']
    weight_slabs = row['Weight Slabs']
    forward_fixed_charge = row['Forward Fixed Charge']
    forward_additional_slab_charge = row['Forward Additional Weight Slab Charge']
    rto_fixed_charge = row['RTO Fixed Charge']
    rto_additional_slab_charge = row['RTO Additional Weight Slab Charge']
    shipment_type = row['Type of Shipment']

    quotent = weight_slabs_by_comp // weight_slabs

    if quotent == 1 and shipment_type == 'Forward charges':
        return forward_fixed_charge

    elif quotent > 1 and shipment_type == 'Forward charges':
        return forward_fixed_charge + forward_additional_slab_charge*(quotent-1)

    elif quotent == 1 and shipment_type == 'Forward and RTO charges':
        return forward_fixed_charge + rto_fixed_charge

    elif quotent > 1 and shipment_type == 'Forward and RTO charges':
        return forward_fixed_charge + forward_additional_slab_charge*(quotent-1) + rto_fixed_charge + rto_additional_slab_charge*(quotent-1)

    return None  

# Apply the custom function to each row to calculate the total forward charges
fwd_df['Total expec Forward Charges'] = fwd_df.apply(calculate_total_forward_charges, axis=1)
fwd_df.head()


,Order ID,total_weight_by_comp,Zone_by_comp,Weight_Slab_by_comp,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge,Type of Shipment,Total expec Forward Charges
0,2001806232,1.302,d,2.50,1.25,45.4,44.8,41.3,44.8,Forward charges,90.2
1,2001806273,0.615,d,1.25,1.25,45.4,44.8,41.3,44.8,Forward charges,45.4
2,2001806408,2.265,d,2.50,1.25,45.4,44.8,41.3,44.8,Forward charges,90.2
3,2001806458,0.700,b,1.00,0.50,33.0,28.3,20.5,28.3,Forward charges,61.3
4,2001807012,0.240,d,1.25,1.25,45.4,44.8,41.3,44.8,Forward charges,45.4


In [16]:
# Merging Total Forward charge with main data
new_df = new_df.merge(right=fwd_df[['Order ID','Total expec Forward Charges']],on='Order ID',how='left')
print(new_df.shape)
new_df.head()

(124, 10)


,AWB Code,Order ID,Customer Pincode,total_weight_by_comp,Weight_by_cour,Zone_by_comp,Zone_by_cour,Weight_Slab_by_comp,Weight_Slab_by_cour,Total expec Forward Charges
0,1091117222124,2001806232,507101,1.302,1.30,d,d,2.50,2.50,90.2
1,1091117222194,2001806273,486886,0.615,1.00,d,d,1.25,1.25,45.4
2,1091117222931,2001806408,532484,2.265,2.50,d,d,2.50,2.50,90.2
3,1091117223244,2001806458,143001,0.700,1.00,b,b,1.00,1.00,61.3
4,1091117229345,2001807012,515591,0.240,0.15,d,d,1.25,1.25,45.4


## Calculaing Courier Service billing value

In [17]:
bill_df = new_df[['Order ID','Weight_by_cour', 'Zone_by_cour','Weight_Slab_by_cour']]
bill_df = bill_df.merge(right=cour_rate,left_on='Zone_by_cour',right_on='Zone',how='left')
bill_df.drop(columns=['Zone'],inplace=True)
bill_df = bill_df.merge(right=cour_inv[['Order ID', 'Type of Shipment']],on='Order ID',how='left')
bill_df.head()

,Order ID,Weight_by_cour,Zone_by_cour,Weight_Slab_by_cour,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge,Type of Shipment
0,2001806232,1.30,d,2.50,1.25,45.4,44.8,41.3,44.8,Forward charges
1,2001806273,1.00,d,1.25,1.25,45.4,44.8,41.3,44.8,Forward charges
2,2001806408,2.50,d,2.50,1.25,45.4,44.8,41.3,44.8,Forward charges
3,2001806458,1.00,b,1.00,0.50,33.0,28.3,20.5,28.3,Forward charges
4,2001807012,0.15,d,1.25,1.25,45.4,44.8,41.3,44.8,Forward charges


In [18]:
def calculate_bill_chagre(row):
    weight_slabs_by_cour = row['Weight_Slab_by_cour']
    weight_slabs = row['Weight Slabs']
    forward_fixed_charge = row['Forward Fixed Charge']
    forward_additional_slab_charge = row['Forward Additional Weight Slab Charge']
    rto_fixed_charge = row['RTO Fixed Charge']
    rto_additional_slab_charge = row['RTO Additional Weight Slab Charge']
    shipment_type = row['Type of Shipment']

    quotent = weight_slabs_by_cour // weight_slabs

    if quotent == 1 and shipment_type == 'Forward charges':
        return forward_fixed_charge

    elif quotent > 1 and shipment_type == 'Forward charges':
        return forward_fixed_charge + forward_additional_slab_charge*(quotent-1)

    elif quotent == 1 and shipment_type == 'Forward and RTO charges':
        return forward_fixed_charge + rto_fixed_charge

    elif quotent > 1 and shipment_type == 'Forward and RTO charges':
        return forward_fixed_charge + forward_additional_slab_charge*(quotent-1) + rto_fixed_charge + rto_additional_slab_charge*(quotent-1)

    return None  

# Apply the custom function to each row to calculate the total forward charges
bill_df['Total_Bill_by_cour'] = bill_df.apply(calculate_bill_chagre, axis=1)
bill_df.head()

,Order ID,Weight_by_cour,Zone_by_cour,Weight_Slab_by_cour,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge,Type of Shipment,Total_Bill_by_cour
0,2001806232,1.30,d,2.50,1.25,45.4,44.8,41.3,44.8,Forward charges,90.2
1,2001806273,1.00,d,1.25,1.25,45.4,44.8,41.3,44.8,Forward charges,45.4
2,2001806408,2.50,d,2.50,1.25,45.4,44.8,41.3,44.8,Forward charges,90.2
3,2001806458,1.00,b,1.00,0.50,33.0,28.3,20.5,28.3,Forward charges,61.3
4,2001807012,0.15,d,1.25,1.25,45.4,44.8,41.3,44.8,Forward charges,45.4


In [19]:
# Merging with the main dataset
new_df = new_df.merge(right=bill_df[['Order ID','Total_Bill_by_cour']],on='Order ID',how='left')
new_df.drop(columns=['Customer Pincode'],inplace=True)
new_df.head()

,AWB Code,Order ID,total_weight_by_comp,Weight_by_cour,Zone_by_comp,Zone_by_cour,Weight_Slab_by_comp,Weight_Slab_by_cour,Total expec Forward Charges,Total_Bill_by_cour
0,1091117222124,2001806232,1.302,1.30,d,d,2.50,2.50,90.2,90.2
1,1091117222194,2001806273,0.615,1.00,d,d,1.25,1.25,45.4,45.4
2,1091117222931,2001806408,2.265,2.50,d,d,2.50,2.50,90.2,90.2
3,1091117223244,2001806458,0.700,1.00,b,b,1.00,1.00,61.3,61.3
4,1091117229345,2001807012,0.240,0.15,d,d,1.25,1.25,45.4,45.4


## Obtaining the difference between the Total expected Forward charges by company x and total bill charged by courier service 

In [20]:
new_df['Diff_expec_bill'] = new_df['Total expec Forward Charges'] - new_df['Total_Bill_by_cour']
new_df.head()

,AWB Code,Order ID,total_weight_by_comp,Weight_by_cour,Zone_by_comp,Zone_by_cour,Weight_Slab_by_comp,Weight_Slab_by_cour,Total expec Forward Charges,Total_Bill_by_cour,Diff_expec_bill
0,1091117222124,2001806232,1.302,1.30,d,d,2.50,2.50,90.2,90.2,0.0
1,1091117222194,2001806273,0.615,1.00,d,d,1.25,1.25,45.4,45.4,0.0
2,1091117222931,2001806408,2.265,2.50,d,d,2.50,2.50,90.2,90.2,0.0
3,1091117223244,2001806458,0.700,1.00,b,b,1.00,1.00,61.3,61.3,0.0
4,1091117229345,2001807012,0.240,0.15,d,d,1.25,1.25,45.4,45.4,0.0


## Creating the summary table

In [21]:
correctly_charged = new_df[new_df['Diff_expec_bill'] == 0]
over_charged = new_df[new_df['Diff_expec_bill'] < 0]
under_charged = new_df[new_df['Diff_expec_bill'] > 0]

correctly_charged_count = len(correctly_charged)
over_charged_count = len(over_charged)
under_charged_count = len(under_charged)

correctly_charged_amount = correctly_charged['Diff_expec_bill'].sum()
over_charged_amount = over_charged['Diff_expec_bill'].sum()
under_charged_amount = under_charged['Diff_expec_bill'].sum()

# Create the table
data = {
    'Description': ['Total Orders where X has been correctly charged', 'Total Orders where X has been overcharged', 'Total Orders where X has been undercharged'],
    'Count': [correctly_charged_count, over_charged_count, under_charged_count],
    'Amount (Rs.)': [correctly_charged_amount, over_charged_amount, under_charged_amount]
}
result_table = pd.DataFrame(data)
print(result_table)


                                       Description  Count  Amount (Rs.)
0  Total Orders where X has been correctly charged     48           0.0
1        Total Orders where X has been overcharged     49       -1274.9
2       Total Orders where X has been undercharged     27         580.8


## Creating final dataframe to be exported in excel

In [22]:
#renaming the dataframe
new_df.rename(columns={'AWB Code':'AWB Number','total_weight_by_comp':'Total weight as per X (KG)','Weight_Slab_by_comp': 'Weight slab as per X (KG)',
    'Weight_by_cour':'Total weight as per Courier Company (KG)', 'Weight_Slab_by_cour':'Weight slab charged by Courier Company (KG)','Zone_by_comp': 'Delivery Zone as per X',
    'Zone_by_cour':'Delivery Zone charged by Courier Company','Total expec Forward Charges':'Expected Charge as per X (Rs.)',
    'Total_Bill_by_cour':'Charges Billed by Courier Company (Rs.) ','Diff_expec_bill':'Difference Between Expected Charges and Billed Charges (Rs.)'},inplace=True)

#rearranging the column order
new_df = new_df[['Order ID', 'AWB Number', 'Total weight as per X (KG)','Weight slab as per X (KG)', 'Total weight as per Courier Company (KG)',
'Weight slab charged by Courier Company (KG)', 'Delivery Zone as per X','Delivery Zone charged by Courier Company','Expected Charge as per X (Rs.)',
'Charges Billed by Courier Company (Rs.) ','Difference Between Expected Charges and Billed Charges (Rs.)']]

In [23]:
new_df.head()

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806232,1091117222124,1.302,2.50,1.30,2.50,d,d,90.2,90.2,0.0
1,2001806273,1091117222194,0.615,1.25,1.00,1.25,d,d,45.4,45.4,0.0
2,2001806408,1091117222931,2.265,2.50,2.50,2.50,d,d,90.2,90.2,0.0
3,2001806458,1091117223244,0.700,1.00,1.00,1.00,b,b,61.3,61.3,0.0
4,2001807012,1091117229345,0.240,1.25,0.15,1.25,d,d,45.4,45.4,0.0


## Exporting the Data set and summary table into excel file 

In [24]:
result_table.to_excel('Summary.xlsx',header=True)
new_df.to_excel('Calculations.xlsx',header=True)
